# Washington State – Tesla Supercharger Spatial Pipeline

End-to-end geospatial analysis notebook.

In [ ]:

# ======================================================
# Imports
# ======================================================
import requests
import json
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines


In [ ]:

# ======================================================
# STEP 1 — DOWNLOAD WA COUNTY POLYGONS (ARCGIS → GEOJSON)
# ======================================================
arcgis_url = "https://gis.dnr.wa.gov/site3/rest/services/Public_Boundaries/WADNR_PUBLIC_Cadastre_OpenData/FeatureServer/11/query"
params = {
    "where": "1=1",
    "outFields": "*",
    "outSR": "4326",
    "f": "json"
}

resp = requests.get(arcgis_url, params=params)
resp.raise_for_status()

features = resp.json()["features"]
geojson_features = []

for feat in features:
    rings = feat["geometry"]["rings"]
    geojson_features.append({
        "type": "Feature",
        "properties": feat["attributes"],
        "geometry": {
            "type": "Polygon",
            "coordinates": [[[pt[0], pt[1]] for pt in ring] for ring in rings]
        }
    })

geojson_path = os.path.expanduser("~/Desktop/wa_counties.geojson")
with open(geojson_path, "w") as f:
    json.dump({"type": "FeatureCollection", "features": geojson_features}, f)

geojson_path


In [ ]:

# ======================================================
# STEP 2 — LOAD & CLEAN SUPERCARGER DATA
# ======================================================
super_path = "/Users/judycheng/Desktop/supercharger in washington state.xls"
df = pd.read_excel(super_path)

df = df[df["State"].str.strip() == "Washington"].copy()
df["City"] = df["Address Part 2"].str.split(",").str[0].str.strip()
df["County"] = df["County"].str.replace("County", "", case=False).str.strip()

city_summary = (
    df.groupby(["City", "County"])
      .size()
      .reset_index(name="Supercharger_Count")
      .sort_values("Supercharger_Count", ascending=False)
)

city_summary.head()


In [ ]:

# ======================================================
# STEP 3 — ADD CITY COORDINATES
# ======================================================
coords = {
    "Seattle": (47.6062, -122.3321),
    "Vancouver": (45.6387, -122.6615),
    "Yakima": (46.6021, -120.5059),
    "Lynnwood": (47.8279, -122.3050),
    "Leavenworth": (47.5962, -120.6615),
    "Spokane": (47.6588, -117.4260),
    "Tacoma": (47.2529, -122.4443),
    "Bellevue": (47.6101, -122.2015),
    "Auburn": (47.3073, -122.2285),
    "Liberty Lake": (47.6756, -117.1183),
}

city_summary["lat"] = city_summary["City"].map(lambda c: coords.get(c, (None, None))[0])
city_summary["lon"] = city_summary["City"].map(lambda c: coords.get(c, (None, None))[1])

summary_path = os.path.expanduser("~/Desktop/WA_supercharger_city_summary.xlsx")
city_summary.to_excel(summary_path, index=False)

summary_path


In [ ]:

# ======================================================
# STEP 4 — CONVERT TO GEODATAFRAME
# ======================================================
gdf_points = gpd.GeoDataFrame(
    city_summary.dropna(subset=["lat", "lon"]),
    geometry=gpd.points_from_xy(city_summary["lon"], city_summary["lat"]),
    crs="EPSG:4326"
)
gdf_points.head()


In [ ]:

# ======================================================
# STEP 5 — LOAD COUNTY SHAPEFILE
# ======================================================
shp_path = os.path.expanduser("~/Desktop/cb_2022_us_county_5m/cb_2022_us_county_5m.shp")
wa_counties = gpd.read_file(shp_path)
wa_counties = wa_counties[wa_counties["STATEFP"] == "53"]

wa_counties = wa_counties.to_crs(epsg=3857)
gdf_points = gdf_points.to_crs(epsg=3857)

wa_counties.head()


In [ ]:

# ======================================================
# STEP 6 — STATEWIDE MAP
# ======================================================
fig, ax = plt.subplots(figsize=(14, 12))
wa_counties.plot(ax=ax, color="#EFEFEF", edgecolor="black")
gdf_points.plot(
    ax=ax,
    markersize=gdf_points["Supercharger_Count"] * 300,
    color="red",
    edgecolor="black",
    alpha=0.75
)

for _, row in gdf_points.iterrows():
    ax.text(row.geometry.x + 6000, row.geometry.y + 6000, row["City"], fontsize=8)

ax.set_title("Washington State – Tesla Supercharger Locations", fontsize=18)
ax.set_axis_off()
plt.show()


In [ ]:

# ======================================================
# STEP 7 — COUNTY MAPS
# ======================================================
output_folder = os.path.expanduser("~/Desktop/WA_county_maps")
os.makedirs(output_folder, exist_ok=True)

for county in wa_counties["NAME"].unique():
    county_geom = wa_counties[wa_counties["NAME"] == county]
    union_geom = county_geom.union_all()

    pts = gdf_points[gdf_points.within(union_geom)]

    fig, ax = plt.subplots(figsize=(10, 8))
    county_geom.plot(ax=ax, color="#EFEFEF", edgecolor="black")

    if len(pts):
        pts.plot(
            ax=ax,
            markersize=pts["Supercharger_Count"] * 300,
            color="red",
            edgecolor="black",
            alpha=0.75
        )

    ax.set_title(f"{county} – Existing Supercharger Points", fontsize=15)
    ax.set_axis_off()
    plt.show()
